In [ ]:
import json
import random
from datetime import datetime

# ==========================================
# 1. THE CORE SYSTEM (Shakti AI)
# ==========================================

class ShaktiAI:
    def __init__(self):
        self.name = "Shakti"
        # In a real app, this acts as your Vector Database (Pinecone/Milvus)
        # It holds the "Truth" so the model doesn't hallucinate.
        self.knowledge_base = {
            "harassment": "Section 75 of BNS (Sexual Harassment) - Punishable up to 3 years.",
            "cyber_stalking": "Section 78 of BNS (Stalking) - Punishable up to 3 years for first conviction.",
            "dowry": "Section 85 of BNS (Cruelty by husband/relatives) - Punishable up to 3 years."
        }
        # This list acts as our "Daily Log" of user conversations
        self.interaction_logs = []

    def get_legal_response(self, user_query):
        """
        Simulates the RAG (Retrieval Augmented Generation) process.
        1. Identifies keywords (Simulated Vector Search).
        2. Fetches correct law from Knowledge Base.
        3. Returns answer.
        """
        query_lower = user_query.lower()

        # Logic to find the relevant law
        found_law = "General Legal Advice - Please contact local police."
        topic_tag = "general"

        if "stalking" in query_lower or "following" in query_lower:
            found_law = self.knowledge_base["cyber_stalking"]
            topic_tag = "stalking"
        elif "harassment" in query_lower or "touching" in query_lower:
            found_law = self.knowledge_base["harassment"]
            topic_tag = "harassment"

        response = f"Namaste. Based on your query, here is the relevant law: {found_law}"

        # LOGGING THE INTERACTION (Crucial for Self-Learning)
        self.log_interaction(user_query, found_law, topic_tag)
        return response

    def log_interaction(self, user_query, system_response, topic):
        """
        Saves the raw data to be processed by the Teacher Model later.
        """
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "user_query": user_query,
            "system_response": system_response,
            "topic": topic,
            "status": "needs_review" # Flag identifying this needs synthetic expansion
        }
        self.interaction_logs.append(log_entry)
        print(f"[System Log]: Interaction saved for topic '{topic}'.")


# ==========================================
# 2. THE TEACHER MODEL (Active Learning)
# ==========================================

class TeacherModel:
    def __init__(self):
        # In reality, this is a larger LLM (like Llama-3-70B or GPT-4)
        # running on a server to generate data overnight.
        self.role = "Data Augmentation Engine"

    def generate_synthetic_data(self, seed_interaction):
        """
        Takes ONE real user query and generates FIVE new variations.
        This allows the model to learn edge cases it hasn't seen yet.
        """
        original_query = seed_interaction['user_query']
        law_context = seed_interaction['system_response']

        print(f"\n[Teacher Model] Generating synthetic data for: '{original_query}'...")

        # Simulating AI generation of new scenarios based on the seed
        # Real code would call an LLM API here.
        synthetic_variations = [
            f"Variation 1: My neighbor is {seed_interaction['topic']} me, what do I do?",
            f"Variation 2: Can I file an FIR if someone is {seed_interaction['topic']} my sister?",
            f"Variation 3: Is {seed_interaction['topic']} a bailable offense in Madhya Pradesh?",
            f"Variation 4: Anonymous {seed_interaction['topic']} on Instagram, help me.",
            f"Variation 5: Police refused to take complaint for {seed_interaction['topic']}.",
        ]

        new_training_data = []
        for var in synthetic_variations:
            # Create a clean Q&A pair for training
            new_training_data.append({
                "instruction": var,
                "output": law_context, # The correct law remains the same
                "source": "synthetic_generated"
            })

        return new_training_data

    def update_dataset(self, new_data):
        """
        Simulates adding this data to the permanent training set.
        """
        # In a real app, this appends to a JSONL file used for fine-tuning Llama
        filename = "shakti_training_data.jsonl"
        with open(filename, "a") as f:
            for item in new_data:
                f.write(json.dumps(item) + "\n")
        print(f"[Dataset Update] Successfully added {len(new_data)} new training examples to {filename}.")


# ==========================================
# 3. RUNNING THE SIMULATION
# ==========================================

if __name__ == "__main__":
    # Initialize Systems
    shakti = ShaktiAI()
    teacher = TeacherModel()

    # --- SCENARIO: Morning (User uses the app) ---
    print("--- PHASE 1: Live User Interaction ---")
    user_input = "Someone is following me and sending bad messages."
    print(f"User: {user_input}")

    # Shakti responds
    reply = shakti.get_legal_response(user_input)
    print(f"Shakti: {reply}")

    # --- SCENARIO: Night (System self-learns) ---
    print("\n--- PHASE 2: Nightly Self-Learning Loop ---")

    # The system picks up today's logs
    todays_logs = shakti.interaction_logs

    for log in todays_logs:
        if log['status'] == "needs_review":
            # 1. Generate new data from this one interaction
            new_synthetic_data = teacher.generate_synthetic_data(log)

            # 2. Update the core dataset
            teacher.update_dataset(new_synthetic_data)

            # 3. Mark log as processed
            log['status'] = "processed"

    print("\n[System Status] Cycle Complete. Shakti is now smarter for tomorrow.")

--- PHASE 1: Live User Interaction ---
User: Someone is following me and sending bad messages.
[System Log]: Interaction saved for topic 'stalking'.
Shakti: Namaste. Based on your query, here is the relevant law: Section 78 of BNS (Stalking) - Punishable up to 3 years for first conviction.

--- PHASE 2: Nightly Self-Learning Loop ---

[Teacher Model] Generating synthetic data for: 'Someone is following me and sending bad messages.'...
[Dataset Update] Successfully added 5 new training examples to shakti_training_data.jsonl.

[System Status] Cycle Complete. Shakti is now smarter for tomorrow.


In [ ]:
# @title Shakti Setu: AI Women's Safety Sentinel (Full System)
variable_name = ""

import os
import subprocess
import sys

# --- AUTO-INSTALL LIBRARIES ---
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import librosa
    import soundfile
    from transformers import pipeline
except ImportError:
    print("⏳ Installing required AI libraries... (This may take 1-2 minutes)")
    install("librosa")
    install("soundfile")
    install("transformers")
    install("scikit-learn")
    install("numpy")
    install("pandas")
    print("✅ Installation Complete.")

import json
import time
import numpy as np
import pandas as pd
import soundfile as sf
import librosa
import warnings
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import pipeline, set_seed

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# ==========================================
# 1. KNOWLEDGE BASE (State-Specific Data)
# ==========================================
STATE_KNOWLEDGE_BASE = {
    "madhya_pradesh": {
        "helpline": "1090 (Women Power Line)",
        "schemes": "Ladli Laxmi Yojana, Gauravi (One Stop Crisis Centre)",
        "laws": "MP Freedom of Religion Act (prevention of forced conversion).",
        "crime_stats": "High reporting of domestic violence in rural districts."
    },
    "delhi": {
        "helpline": "181 (Abhayam)",
        "schemes": "Himmat Plus App, Pink Ticket (Free Bus Travel)",
        "laws": "Specific focus on POCSO and street harassment laws.",
        "crime_stats": "High incidence of stalking and snatching."
    },
    "maharashtra": {
        "helpline": "103 (Special Women's Helpline)",
        "schemes": "Manodhairya Scheme (Compensation for victims)",
        "laws": "Shakti Act (Proposed) - Death penalty for heinous rape cases.",
        "crime_stats": "Cyber-stalking cases rising in Mumbai/Pune."
    }
}

EMERGENCY_CONTACTS = {
    "POLICE": "112",
    "FIRE": "101",
    "AMBULANCE": "108",
    "FAMILY": "+91-9876543210"
}

# ==========================================
# 2. SHAKTI BRAIN (Text & Active Learning)
# ==========================================
class ShaktiBrain:
    def __init__(self):
        print("[Init] Loading Language Model (DistilGPT2)...")
        self.generator = pipeline('text-generation', model='distilgpt2')
        set_seed(42)
        self.learning_log = []

    def query(self, user_text, state="madhya_pradesh"):
        state_key = state.lower().replace(" ", "_")
        data = STATE_KNOWLEDGE_BASE.get(state_key, STATE_KNOWLEDGE_BASE["madhya_pradesh"])

        context = (f"You are Shakti, a helpful Indian legal assistant for women. "
                   f"The user is in {state}. Local Helpline is {data['helpline']}. "
                   f"Key Schemes: {data['schemes']}. "
                   f"User Query: {user_text}")

        response = (f"Namaste. Since you are in {state.title()}, help is available.\n"
                    f"🚨 **Emergency Helpline:** {data['helpline']}\n"
                    f"📜 **Relevant Scheme:** {data['schemes']}\n"
                    f"⚖️ **Legal Insight:** {data['laws']}")

        self.active_learning_loop(user_text, state)
        return response

    def active_learning_loop(self, original_query, state):
        print(f"\n[Active Learning] 🧠 Learning from: '{original_query}'...")
        prompt = f"List 3 different ways a woman might ask for help regarding '{original_query}':"
        try:
            variations = self.generator(prompt, max_length=60, num_return_sequences=1)[0]['generated_text']
            clean_variations = variations.split('\n')[1:]
        except:
            clean_variations = ["Variation generation failed - Low Memory"]

        new_entry = {
            "timestamp": datetime.now().isoformat(),
            "state": state,
            "original": original_query,
            "synthetic_variations": clean_variations
        }
        self.learning_log.append(new_entry)
        print(f"[Active Learning] ✅ Generated {len(clean_variations)} new synthetic training samples.")


# ==========================================
# 3. SHAKTI EAR (Audio Distress Detection)
# ==========================================
class ShaktiEar:
    def __init__(self):
        print("[Init] Training Distress Detection Audio Model...")
        self.model = RandomForestClassifier(n_estimators=50)
        self.train_dummy_model()

    def generate_synthetic_audio_features(self, n_samples=100):
        # Simulate MFCC features: Class 0 (Normal) vs Class 1 (High Pitch/Distress)
        X_normal = np.random.normal(loc=-20, scale=5, size=(n_samples, 40))
        y_normal = np.zeros(n_samples)
        X_distress = np.random.normal(loc=10, scale=10, size=(n_samples, 40))
        y_distress = np.ones(n_samples)
        X = np.vstack((X_normal, X_distress))
        y = np.hstack((y_normal, y_distress))
        return X, y

    def train_dummy_model(self):
        X, y = self.generate_synthetic_audio_features()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        self.model.fit(X_train, y_train)
        acc = accuracy_score(y_test, self.model.predict(X_test))
        print(f"[Init] Audio Model Trained. Accuracy: {acc*100:.1f}%")

    def simulate_listening(self, text_input):
        """
        Simulates the pipeline:
        1. Keywords Check (What she said)
        2. Tone Analysis (How she said it - using ML Model)
        """
        trigger_map = {
            "bachao": "POLICE", "help": "POLICE", "bacho": "POLICE",
            "aag": "FIRE", "fire": "FIRE",
            "chot": "AMBULANCE", "hurt": "AMBULANCE", "khoon": "AMBULANCE", "blood": "AMBULANCE"
        }

        # 1. KEYWORD CHECK
        detected_intent = None
        for word in text_input.lower().split():
            if word in trigger_map:
                detected_intent = trigger_map[word]
                break

        # 2. TONE ANALYSIS (Simulating Audio Input)
        # In a real app, we would extract MFCC from the microphone here.
        # For simulation: If user types "!", we assume High Pitch/Screaming audio.
        if "!" in text_input:
            # Generate synthetic "Distress" audio features
            simulated_audio_features = np.random.normal(loc=10, scale=10, size=(1, 40))
        else:
            # Generate synthetic "Normal" audio features
            simulated_audio_features = np.random.normal(loc=-20, scale=5, size=(1, 40))

        tone_prediction = self.model.predict(simulated_audio_features)[0] # 1 = Distress, 0 = Normal

        print(f"\n[Analysis] Keywords: {'Matched' if detected_intent else 'None'}")
        print(f"[Analysis] Voice Tone (ML Prediction): {'High Pitch/Panic' if tone_prediction == 1 else 'Normal'}")

        # 3. DECISION LOGIC
        if detected_intent:
            self.trigger_emergency(detected_intent)
        elif tone_prediction == 1:
            print("⚠️ High Distress Tone detected, but words unclear. Alerting Police as precaution.")
            self.trigger_emergency("POLICE")
        else:
            print("✅ Voice processed. No emergency detected.")

    def trigger_emergency(self, service):
        print(f"\n🚨🚨 DISTRESS DETECTED: {service} 🚨🚨")
        print(f"📞 Auto-Dialing {EMERGENCY_CONTACTS[service]}...")
        print(f"📍 Sending Live Location to Family Contact...")

# ==========================================
# 4. MAIN EXECUTION
# ==========================================

shakti_brain = ShaktiBrain()
shakti_ear = ShaktiEar()

print("\n" + "="*40)
print("   🙏 SHAKTI SETU IS ONLINE 🙏")
print("="*40)

while True:
    print("\nSelect Mode:")
    print("1. Text Query (Women's Rights/Laws)")
    print("2. Simulate Voice Distress Call")
    print("3. View Active Learning Logs")
    print("4. Exit")

    choice = input("Enter choice (1-4): ")

    if choice == '1':
        user_state = input("Enter your State (e.g., Madhya Pradesh, Delhi): ")
        user_q = input("Enter your problem (e.g., My husband beats me): ")
        print("\n" + "-"*30)
        print(shakti_brain.query(user_q, user_state))
        print("-"*30)

    elif choice == '2':
        print("\n🎤 Listening Mode Active... (Simulated)")
        print("💡 Tip: Add '!' to simulate SCREAMING (e.g., 'Bachao!'). Normal typing simulates calm voice.")
        voice_input = input("User Input: ")
        shakti_ear.simulate_listening(voice_input)

    elif choice == '3':
        print("\n🧠 [System Internals] New Data Generated:")
        for log in shakti_brain.learning_log:
            print(json.dumps(log, indent=2))

    elif choice == '4':
        print("Shutting down Shakti Setu. Stay Safe.")
        break
    else:
        print("Invalid choice.")

[Init] Loading Language Model (DistilGPT2)...


Device set to use cpu


[Init] Training Distress Detection Audio Model...
[Init] Audio Model Trained. Accuracy: 100.0%

   🙏 SHAKTI SETU IS ONLINE 🙏

Select Mode:
1. Text Query (Women's Rights/Laws)
2. Simulate Voice Distress Call
3. View Active Learning Logs
4. Exit
Enter choice (1-4): 1
Enter your State (e.g., Madhya Pradesh, Delhi): delhi
Enter your problem (e.g., My husband beats me): My husband beats me


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



------------------------------

[Active Learning] 🧠 Learning from: 'My husband beats me'...
[Active Learning] ✅ Generated 7 new synthetic training samples.
Namaste. Since you are in Delhi, help is available.
🚨 **Emergency Helpline:** 181 (Abhayam)
📜 **Relevant Scheme:** Himmat Plus App, Pink Ticket (Free Bus Travel)
⚖️ **Legal Insight:** Specific focus on POCSO and street harassment laws.
------------------------------

Select Mode:
1. Text Query (Women's Rights/Laws)
2. Simulate Voice Distress Call
3. View Active Learning Logs
4. Exit
Enter choice (1-4): 
